In [8]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import os
import random

from sklearn.metrics import recall_score as recall
from sklearn.metrics import precision_score as precision
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

#from keras.utils import np_utils

In [2]:
file_dir = "C:/Users/ramse/Documents/School/Northeastern/Machine Learning/Final Project/OneDrive_1_4-6-2022/"
labels = pd.read_csv(file_dir + 'hemorrhage-labels.csv',index_col=None)
#display(labels)
#y = labels.CDR
count = list()
for i,j,k,l,m in zip(labels['epidural'],labels['intraparenchymal'],labels['intraventricular'],labels['subarachnoid'],labels['subdural']):
    if i==1:
        count.append(1)
    elif j==1:
        count.append(2)
    elif k==1:
        count.append(3)
    elif l==1:
        count.append(4)
    elif m==1:
        count.append(5)
    else:
        count.append(0)

print("epidural is assigned as 1")
print("intraparenchymalis assigned as 2")
print("intraventricular is assigned as 3")
print("subarachnoid is assigned as 4")
print("subdural is assigned as 5")
print("0 indicates no hemorrhage")

labels["HemorrageType"]=count

display(labels)

epidural is assigned as 1
intraparenchymalis assigned as 2
intraventricular is assigned as 3
subarachnoid is assigned as 4
subdural is assigned as 5
0 indicates no hemorrhage


,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,HemorrageType
0,ID_000012eaf,0,0,0,0,0,0,0
1,ID_000039fa0,0,0,0,0,0,0,0
2,ID_00005679d,0,0,0,0,0,0,0
3,ID_00008ce3c,0,0,0,0,0,0,0
4,ID_0000950d7,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
752798,ID_ffff82e46,0,0,0,0,0,0,0
752799,ID_ffff922b9,1,0,0,1,0,0,3
752800,ID_ffffb670a,1,0,0,0,1,0,4
752801,ID_ffffcbff8,0,0,0,0,0,0,0


In [3]:
cttype = ['brain_bone_window/','brain_window/','max_contrast_window/','subdural_window/']

In [4]:
def downsample(downSampleRate, image):
    im_size = image.size
    #N_train = y_train.shape[0]  # The length of the training data

    if im_size/downSampleRate % 1 > 0:
        print("Downsample rate is not a multiple of", im_size)
        return [-1]
    else:
        im_size = int(im_size/downSampleRate)
        
    return image[0::downSampleRate]

In [5]:
def getListOfFiles(dirName, ctype):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    justNames = list()
    # Iterate over all the entries
    for entry in listOfFile:
        
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            a, b = getListOfFiles(fullPath, ctype)
            allFiles = allFiles + a
            justNames = justNames + b
        elif ctype in fullPath:
            allFiles.append(fullPath)
            justNames.append(entry)
                
    return allFiles, justNames

In [6]:
a,b = getListOfFiles('C:/Users/ramse/Documents/School/Northeastern/Machine Learning/Final Project/OneDrive_1_4-6-2022/renders/', 'max_contrast')

#Sort Alphanumerically to match csv file
x = np.array(b)
ind = np.argsort(x)
print(ind)
y = np.array(a)

filenamesSorted = x[ind]
filePathsSorted = y[ind]

labels.sort_values(by=['Image'])

print(filenamesSorted[0:20], filenamesSorted[-20:-1])
#labels

#labels["fullFileName"] = y[ind]

[59310 59311 59312 ... 59309 27235 84331]
['ID_000012eaf.jpg' 'ID_000039fa0.jpg' 'ID_00005679d.jpg'
 'ID_00008ce3c.jpg' 'ID_0000950d7.jpg' 'ID_0000aee4b.jpg'
 'ID_0000ca2f6.jpg' 'ID_0000f1657.jpg' 'ID_000178e76.jpg'
 'ID_00019828f.jpg' 'ID_0001dcc25.jpg' 'ID_0001de0e8.jpg'
 'ID_0001f7f62.jpg' 'ID_0002003a8.jpg' 'ID_0002081b6.jpg'
 'ID_0002108bd.jpg' 'ID_000229f2a.jpg' 'ID_000230ed7.jpg'
 'ID_000259ccf.jpg' 'ID_00025ef4b.jpg'] ['ID_fff4c8969.jpg' 'ID_fff4dc2f1.jpg' 'ID_fff5ad8a5.jpg'
 'ID_fff60ea80.jpg' 'ID_fff683ffd.jpg' 'ID_fff6e7944.jpg'
 'ID_fff7a1309.jpg' 'ID_fff882c03.jpg' 'ID_fff92ad89.jpg'
 'ID_fff974da8.jpg' 'ID_fff9bb57a.jpg' 'ID_fffa58f70.jpg'
 'ID_fffaebafd.jpg' 'ID_fffb124e6.jpg' 'ID_fffb16e96.jpg'
 'ID_fffc60817.jpg' 'ID_fffd00949.jpg' 'ID_fffe2edb8.jpg'
 'ID_ffff922b9.jpg']


In [9]:
#saturncloud.io

downSampleRate = 16 # Set to 1 for full data set
numDataPoints =  10000 # Set to len(filenamesSorted)
indices = random.sample(range(0, len(filenamesSorted)-1), numDataPoints)
ind = np.sort(np.array(indices))
print(ind)
x_names = filenamesSorted[ind]
x_data = filePathsSorted[ind]

#print(x_data[0:20], x_data[-20:-1])

im_size = int((512*512)/downSampleRate)
data = np.zeros([numDataPoints, im_size])

dataFrameIter=0
dataSize_iter = 0
dataFrameIndicesInUse = []
for file_name in x_data:
    #print(file_name,labels.Image[dataFrameIter])
    while file_name.find(labels.Image[dataFrameIter]) == -1:
        #print(file_name,labels.Image[dataFrameIter])
        dataFrameIter+=1
    
    dsImage = downsample(16, np.mean(matplotlib.image.imread(file_name),axis=2).reshape(-1))
    if len(dsImage) == im_size:
        data[0,:] = dsImage
        dataSize_iter+=1
        dataFrameIndicesInUse.append(dataFrameIter)


[     0      7     19 ... 116493 116495 116523]
Downsample rate is not a multiple of 312620
Downsample rate is not a multiple of 213444
Downsample rate is not a multiple of 213444


In [10]:
#data_save = data
#print(type(data_save))
dataFlat = np.array(data, order='F')
dataFlat.resize((dataSize_iter, im_size))

y = labels["HemorrageType"]
y = y[dataFrameIndicesInUse]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataFlat, y, test_size=0.2, random_state=0)
print(y_train.shape, y_test.shape)
print(X_train.shape, X_test.shape)

(7992,) (1998,)
(7992, 16384) (1998, 16384)


In [11]:
# Covert Y to be categorical
print(y_test.unique())


[5 4 3 0 2 1]


In [12]:


## Logistic Regression
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=5000)
clf.fit(X_train,y_train)


LogisticRegression(max_iter=5000, multi_class='multinomial')

In [11]:

print("Logistic Regression Score: %.3f"%clf.score(X_test,y_test))
y_pred = clf.predict(X_test)
con_mat = confusion_matrix(y_pred,y_test)
print("Confusion Matrix:\n",con_mat)
print("Recall: ",recall(y_pred, y_test, average=None))
print("Precision: ",precision(y_pred, y_test, average=None))

Logistic Regression Score: 0.299
Confusion Matrix:
 [[  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [129  42 598 274 394 562]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]]
Recall:  [0.         0.         0.29914957 0.         0.         0.        ]
Precision:  [0. 0. 1. 0. 0. 0.]


C:\Users\ramse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:


## Linear Discriminant Analysis

lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X_train, y_train)


LinAlgError: SVD did not converge

In [13]:


print("LDA Score: %.3f"%lda.score(X_test,y_test))





AttributeError: 'LinearDiscriminantAnalysis' object has no attribute 'coef_'

In [14]:
y_pred = lda.predict(X_test)

AttributeError: 'LinearDiscriminantAnalysis' object has no attribute 'coef_'

In [15]:
con_mat = confusion_matrix(y_pred,y_test)
print("Confusion Matrix:\n",con_mat)

Confusion Matrix:
 [[  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [129  42 598 274 394 562]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]]


In [16]:
print("Recall: ",recall(y_pred, y_test, average=None))

Recall:  [0.         0.         0.29914957 0.         0.         0.        ]


C:\Users\ramse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
print("Precision: ",precision(y_pred, y_test, average=None))

Precision:  [0. 0. 1. 0. 0. 0.]


In [18]:
## Quadratic Discriminant Analysis



qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_train, y_train)


C:\Users\ramse\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(store_covariance=True)

In [22]:
print("QDA Score: %.3f"%qda.score(X_test,y_test))
y_pred = qda.predict(X_test)
con_mat = confusion_matrix(y_pred,y_test)
print("Confusion Matrix:\n",con_mat)
print("Recall: ",recall(y_pred, y_test, average=None))
print("Precision: ",precision(y_pred, y_test, average=None))

QDA Score: 0.139
Confusion Matrix:
 [[  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   5   2   3   2]
 [127  41 590 270 391 558]
 [  2   1   2   1   0   0]
 [  0   0   1   1   0   2]]
Recall:  [0.         0.         0.41666667 0.13657056 0.         0.5       ]
Precision:  [0.         0.         0.0083612  0.98540146 0.         0.00355872]


C:\Users\ramse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:


lr = LinearRegression()
lr.fit(X_train, pd.get_dummies(y_train))


LinearRegression()

In [21]:
y_hat = np.argmax(lr.predict(X_test), axis=1)
y_true = np.argmax(np.matrix(pd.get_dummies(y_test)), axis=1)
print("Linear Regression Score: %.3f"%accuracy_score(y_hat,y_true))
con_mat = confusion_matrix(y_hat,y_true)
correct_percent = (con_mat[0][0]+con_mat[1][1])/(sum(con_mat[:][0])+sum(con_mat[:][1]))*100
print("Confusion Matrix:\n",con_mat)
print("The linear model predicted the right category of voice data %.2f%% of the time"%(correct_percent))
print("Recall: ",recall(y_hat, y_true, average=None))
print("Precision: ",precision(y_hat, y_true, average=None))

Linear Regression Score: 0.137
Confusion Matrix:
 [[  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [129  42 598 274 394 562]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]]
The linear model predicted the right category of voice data nan% of the time
Recall:  [0.         0.         0.         0.13706853 0.         0.        ]
Precision:  [0. 0. 0. 1. 0. 0.]


C:\Users\ramse\AppData\Local\Temp/ipykernel_17928/1016897161.py:5: RuntimeWarning: invalid value encountered in longlong_scalars
  correct_percent = (con_mat[0][0]+con_mat[1][1])/(sum(con_mat[:][0])+sum(con_mat[:][1]))*100
C:\Users\ramse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
